In [1]:
import os 
os.chdir(r'E:\Study_Related_Stuff\Git repo\Machine-Learning\CTC_prediction')

In [2]:
import numpy as np
import pandas as pd

In [3]:
mtr=pd.read_csv("train.csv",header=0)
mte=pd.read_csv("test.csv",header=0)

In [4]:
print(mtr.shape)
mtr.head()
mte.shape

(26415, 34)


(10875, 33)

In [5]:
print(mtr.duplicated().sum())
print(mte.duplicated().sum())

print(mtr.isnull().sum().sum())
print(mte.isna().sum().sum())


0
0
0
0



mtr=mtr.replace(-1,np.nan)
mtr=mtr.replace(0,np.nan)
mtr=mtr.replace(-100,0)

mte=mte.replace(-1,np.nan)
mte=mte.replace(0,np.nan)
mte=mte.replace(-100,0)


print(mtr.isnull().sum())

mtr=mtr.fillna(mtr.mode())
mte=mte.fillna(mte.mode())
print(mte.isna().sum())
print(mtr.isna().sum())

In [6]:
mtr=mtr.drop(columns=['CityTier','Candidate ID','Date Of Birth'])
mte=mte.drop(columns=['CityTier','Candidate ID','Date Of Birth'])



In [7]:
mtr.dtypes

Pay_in_INR                        float64
Gender                             object
Score in Tenth                    float64
School Board in Tenth              object
Year Of Twelth Completion           int64
Score in Twelth                   float64
Board in Twelth                    object
CollegeCode                         int64
CollegeTier                         int64
Graduation                         object
Discipline                         object
GPA Score in Graduation           float64
CityCode                            int64
State                              object
Year of Graduation Completion       int64
Score in English language           int64
Score in Logical skill              int64
Score in Quantitative ability       int64
Score in Domain                   float64
Score in ComputerProgramming        int64
Score in ElectronicsAndSemicon      int64
Score in ComputerScience            int64
Score in MechanicalEngg             int64
Score in ElectricalEngg           

In [8]:
x=mtr.drop(columns=['Pay_in_INR'])
y=mtr['Pay_in_INR']

In [9]:
cat=[col for col in x.columns if x[col].dtype == 'object']
print(cat)

testw=[colu for colu in mte.columns if mte[colu].dtype=='object']
testw

['Gender', 'School Board in Tenth', 'Board in Twelth', 'Graduation', 'Discipline', 'State']


['Gender',
 'School Board in Tenth',
 'Board in Twelth',
 'Graduation',
 'Discipline',
 'State']

In [10]:
x=pd.get_dummies(x,columns =cat)
test=pd.get_dummies(mte,columns=testw)

from sklearn.datasets import load_digits
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import chi2,SelectKBest

y=mtr['Pay_in_INR']
x=mtr.drop(columns='Pay_in_INR')

sigfea=SelectKBest(score_func=f_regression,k=10)
fits = sigfea.fit(x,y)
chi_support=sigfea.get_support()
chi_features = x.loc[:,chi_support].columns.tolist()
print(chi_features)

In [11]:
from sklearn import preprocessing as pps
std=pps.StandardScaler().fit(x[:]).transform(x[:])
x.head()

from sklearn import preprocessing as pps
std=pps.StandardScaler().fit(test[:]).transform(test[:])
test.head() 

,Score in Tenth,Year Of Twelth Completion,Score in Twelth,CollegeCode,CollegeTier,GPA Score in Graduation,CityCode,Year of Graduation Completion,Score in English language,Score in Logical skill,...,State_Meghalaya,State_Orissa,State_Punjab,State_Rajasthan,State_Sikkim,State_Tamil Nadu,State_Telangana,State_Uttar Pradesh,State_Uttarakhand,State_West Bengal
0,78.00,2008,76.80,11368,2,73.90,11368,2012,545,625,...,False,False,False,False,False,False,False,True,False,False
1,89.92,2010,87.00,5086,2,76.32,5086,2014,560,555,...,False,False,False,False,False,False,False,False,False,False
2,60.80,2006,66.66,41,2,74.74,41,2013,465,585,...,False,False,False,False,False,False,False,False,False,False
3,88.00,2010,90.00,9173,2,76.00,9173,2014,625,555,...,False,False,False,False,False,False,True,False,False,False
4,86.80,2008,82.80,9019,2,78.00,9019,2012,575,585,...,False,False,False,False,False,False,False,False,False,False


In [12]:
print(x.shape)
print(test.shape)

(26415, 694)
(10875, 290)


In [13]:
r=[c for c in x.columns  if c in test.columns]
f=x[r]
x=pd.DataFrame(f)

In [14]:
x.shape

(26415, 290)

In [15]:
test.shape

(10875, 290)

# Splitting and Model building 

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=20)       

In [17]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
print(test.shape)
print(type(x_train),type(x_test),type(y_train),type(y_test))


(19811, 290) (19811,) (6604, 290) (6604,)
(10875, 290)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [18]:
from sklearn import tree 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
deci=tree.DecisionTreeRegressor() # ,max_depth=3 criterion='gini'
deci_fit=deci.fit(x_train,y_train)
params = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
          'max_depth': [i for i in range(1,10)], 
          #'n_estimators' : [50.100,150,200,250,400,500],
          'random_state':[123]}
rf_random = RandomizedSearchCV(estimator = deci, param_distributions = params, n_iter = 100, cv = 3, verbose=0, random_state=42)
rf_random.fit(x_train,y_train)
#print('Best Penalty:', rf_random.best_estimator_.get_params())
rf_random.best_params_ 
rf_random.best_estimator_

D:\all\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
105 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
105 fits failed with the following error:
Traceback (most recent call last):
  File "D:\all\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\all\lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "D:\all\lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "D:\all\lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_parameter_constraints
    rai

DecisionTreeRegressor(max_depth=9, max_features='sqrt', min_samples_leaf=9,
                      min_samples_split=15, random_state=123)

In [19]:
from sklearn.metrics import mean_squared_error 
from sklearn.tree import DecisionTreeRegressor

deci1=tree.DecisionTreeRegressor(max_depth=8, max_features='sqrt', min_samples_leaf=2,              min_samples_split=9, random_state=123)
deci_fit=deci1.fit(x_train,y_train)

deci_31 = deci1.predict(test)

pre_test=deci1.predict(x_test)
deci_ =mean_squared_error(y_test,pre_test) 
deci_ 

90873898480.64049

In [20]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

clf = xgb.XGBRegressor()
param_grid = {
        
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [50]}

xgboo = RandomizedSearchCV(clf, param_grid, n_iter=10,
                            n_jobs=2, verbose=1, cv=4, random_state=42)
xgboo.fit(x_train,y_train)
xgboo.best_params_ 
xgboo.best_estimator_

Fitting 4 folds for each of 10 candidates, totalling 40 fits


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=0.8, colsample_bynode=None, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.25, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=20, max_leaves=None,
             min_child_weight=1.0, missing=nan, monotone_constraints=None,
             n_estimators=50, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [21]:
print('xgboo.best_params_ ',xgboo.best_params_ )

xgboo.best_params_  {'subsample': 0.6, 'reg_lambda': 1.0, 'n_estimators': 50, 'min_child_weight': 1.0, 'max_depth': 20, 'learning_rate': 0.1, 'gamma': 0.25, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.8}


In [22]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb
xgbo=xgb.XGBRegressor(n_estimators=100,subsample= 0.6,
 reg_lambda= 1.0,
 
 min_child_weight=1.0,
 max_depth=20,
 learning_rate= 0.1,
 gamma= 0.25,
 colsample_bytree= 0.8,
 colsample_bylevel= 0.8 )
xgbo.fit(x_train,y_train)

xg_pre=xgbo.predict(test)
x_pre=xgbo.predict(x_test)
xgb_mse='xgboost acc',mean_squared_error(y_test,x_pre)
xgb_mse 

('xgboost acc', 19198871710.737896)

In [23]:
from sklearn.ensemble import RandomForestRegressor

model3=RandomForestRegressor() 

para_rf= { 'max_depth':([i for i in range(1,12)]),
                      'max_features':["auto", "sqrt","log2"], 'max_leaf_nodes':([j for j in range(1,12)]),
                      
                       'min_samples_split':([l for l in range(1,12)])}
ran_rf=RandomizedSearchCV(model3,para_rf,random_state=14,n_jobs=-1,cv=4,verbose=2)


In [24]:

ran_rf.fit(x_train,y_train)
ran_rf.best_params_
ran_rf.best_estimator_

Fitting 4 folds for each of 10 candidates, totalling 40 fits


D:\all\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
8 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "D:\all\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\all\lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "D:\all\lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "D:\all\lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_parameter_constraints
    raise In

RandomForestRegressor(max_depth=5, max_features='sqrt', max_leaf_nodes=10,
                      min_samples_split=3)

In [ ]:
from sklearn import svm 
from sklearn.model_selection import RandomizedSearchCV
model=svm.SVC()
para={'kernel':['rbf','linear'],'C':[0.001,0.1,0.01,1.0,10.0,100.0,2.0], 'gamma':([i for i in range(1,3)],0.1)}
ra_svm=RandomizedSearchCV(model,para,cv=3,random_state=15,verbose=1)
ra_svm.fit(x_train,y_train)
ra_svm.best_params_  
ra_svm.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


D:\all\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


In [ ]:
from sklearn import svm
svc = svm.SVC(kernel='linear', C=1,gamma=1).fit(x_train, y_train)
svc_pre=svc.predict(test)
val_pre=svc.predict(x_test)
svm_score=mean_sqauared_error(y_test,val_pre)
svm_score